# 03_BT_01_Betas_v1

This notebook calculates the __first version__ of the betas (Value given from people in interviews to physical variables in the analysed territories)

## Import libraries

In [1]:
first_folder_path = '../'

In [2]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import src
else:
    import src

## Notebook config

## Input - Load surveys

In [3]:
# Read file (Converted from .xlsx to .csv UTF-8)
#'Encuesta_caminata(Guadalajara)(1-347).csv'
#'Encuesta_caminata(Guadalajara)(1-510).csv'
#'Encuesta_caminata(Guadalajara)(1-553).csv'

surveys_file = 'Encuesta_caminata(Guadalajara)(1-553).csv'
surveys = pd.read_csv(first_folder_path + f"data/vref/output/table/encuestas/{surveys_file}")

# Show
surveys.head(2)

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
0,1,6/28/24 12:24:50,6/28/24 12:25:46,anonymous,NaN,NaN,DS,6/25/2024,001,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
1,2,6/28/24 12:25:58,6/28/24 12:28:18,anonymous,NaN,NaN,DS,6/25/2024,002,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [4]:
surveys.columns

Index(['ID', 'Hora de inicio', 'Hora de finalización', 'Correo electrónico',
       'Nombre', 'Hora de la última modificación', 'Encuestador',
       'Fecha de realización', 'Encuesta #', 'Territorio de análisis',
       'Lugar donde se realizó la encuesta', 'Género', 'Edad (años)',
       'Estrato socioeconómico (NO APLICA)', 'Ocupación principal',
       'Nivel educativo completado',
       '¿Poseen alguno de los siguientes vehículos en su vivienda?',
       'Indique su lugar de residencia (Municipio/comuna/barrio)',
       '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?',
       '¿Cómo suple las necesidades de equipamientos educativos?',
       '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?',
       '¿Cuáles?',
       '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?',
       '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
       'En una semana típica, ¿Cuántas veces 

## Filter for data of interest

In [5]:
transport_mode_question = '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)'
surveys[transport_mode_question].unique()

array(['Uber', 'Carro particular - coche', 'Caminata', 'Mototaxi',
       'SITVA (Metro, Metrocable, Metroplus, Tranvía, Bus alimentador)',
       'Motocicleta', 'Transporte Centro de Rehabilitación (Arenales)',
       'Bus', 'Plataformas digitales (Uber, Didi, InDriver, Picap, etc.)',
       'Taxi', 'Transporte privado de parte de la empresa donde labora',
       'Bicicleta', 'Transporte empresarial', 'Patineta eléctrica',
       'Ride', 'Transporte privado'], dtype=object)

In [14]:
surveys_f = surveys.loc[surveys[transport_mode_question]=='Caminata'].copy()

# Show
print(surveys_f.shape)
surveys_f.head(2)

(215, 65)


,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
3,4,6/28/24 12:31:03,6/28/24 12:40:00,anonymous,NaN,NaN,DS,6/25/2024,004,TA_Miramar,...,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo
7,8,6/28/24 12:48:51,6/28/24 12:57:14,anonymous,NaN,NaN,DS,6/25/2024,008,TA_Miramar,...,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo


In [20]:
print(f"Colinas {len(surveys_f.loc[surveys_f['Territorio de análisis']=='TA_Colinas'])}.")
print(f"Miramar {len(surveys_f.loc[surveys_f['Territorio de análisis']=='TA_Miramar'])}.")
print(f"Providencia {len(surveys_f.loc[surveys_f['Territorio de análisis']=='TA_Providencia'])}.")

Colinas 14.
Miramar 129.
Providencia 72.


## Rename columns

In [7]:
# INFO - Columns that hold relation with physical variables.

# POPULATION DENSITY
# 'Column' --> 36. (A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?)
# Sí --> PRESENCIA de variable (Hay densidad)
# No --> AUSENCIA de variable (No hay densidad)

# INTERSECTIONS DENSITY
# '2' --> 37. (A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?)
# Sí --> PRESENCIA de variable (Hay densidad de intersecciones)
# No --> AUSENCIA de variable (Faltan intersecciones)

# '3' --> 38. (¿Se te hace largo y aburrido el recorrido?)
# Sí --> AUSENCIA de variable (Faltan intersecciones para diversificar el recorrido)
# No --> PRESENCIA de variable (Hay intersecciones que diversifican el recorrido)

# '4' --> 39. (A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?)
# Sí --> PRESENCIA de variable (Hay densidad de intersecciones)
# No --> AUSENCIA de variable (Faltan intersecciones)

# LAND USE
# '5' --> 40. (A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?)
# Sí --> PRESENCIA de variable (Hay varios usos)
# No --> AUSENCIA de variable (Faltan usos)

# '6' --> 41. (A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?)
# Sí --> PRESENCIA de variable (Hay varias personas realizando actividades)
# No --> AUSENCIA de variable (La cantidad de personas/actividades no influencía la comodidad al caminar)

# '7' --> 42. (A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?)
# Sí --> PRESENCIA de variable (Hay varios usos)
# No --> AUSENCIA de variable (Faltan usos)

# VEGETATION INDEX
# '8' --> 43. (A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?)
# Sí --> PRESENCIA de variable (Hay vegetación)
# No --> AUSENCIA de variable (Falta vegetación)

# '9' --> 44. (A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?)
# Sí --> PRESENCIA de variable (Hay vegetación)
# No --> AUSENCIA de variable (Falta vegetación)

# SLOPE
# '10' --> 45. (A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?)
# Sí --> PRESENCIA de variable (Hay pendiente)
# No --> AUSENCIA de variable (No hay pendiente)

# SIDEWALK PRESENCE
# '11' --> 46. (A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?)
# Sí --> PRESENCIA de variable (La infraestructura te hace sentir seguro)
# No --> AUSENCIA de variable (La infraestructura no es suficiente para hacerte sentir seguro)

# '11' --> 47. (A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?)
# Sí --> PRESENCIA de variable (La infraestructura te permite andar cómodamente)
# No --> AUSENCIA de variable (La infraestructura no es suficiente para permitirte andar cómodamente)

In [8]:
# Tested different survey files, column name unification:

if (surveys_file == 'Encuesta_caminata(Guadalajara)(1-347).csv') or (surveys_file == 'Encuesta_caminata(Guadalajara)(1-510).csv'):
    # Filter for data of interest
    surveys_f = surveys_f[['ID','Territorio de análisis',
                         '36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?',
                         '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?',
                         '38. ¿Se te hace largo y aburrido el recorrido?',
                         '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?',
                         '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?',
                         '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?',
                         '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?',
                         '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?',
                         '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?',
                         '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?',
                         '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?',
                         '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?'
                        ]]
    # Rename columns
    surveys_f.rename(columns={'Territorio de análisis':'territory',
                              '36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?':'36_text',
                              '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?':'37_text',
                              '38. ¿Se te hace largo y aburrido el recorrido?':'38_text',
                              '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?':'39_text',
                              '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?':'40_text',
                              '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?':'41_text',
                              '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?':'42_text',
                              '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?':'43_text',
                              '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?':'44_text',
                              '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?':'45_text',
                              '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?':'46_text',
                              '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?':'47_text'
                             },inplace=True
                    )

elif surveys_file == 'Encuesta_caminata(Guadalajara)(1-553).csv':
    # Filter for data of interest
    surveys_f = surveys_f[['ID','Territorio de análisis','Column', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]
    
    # Rename columns
    surveys_f.rename(columns={'Territorio de análisis':'territory',
                              'Column':'36_text',
                              '2':'37_text',
                              '3':'38_text',
                              '4':'39_text',
                              '5':'40_text',
                              '6':'41_text',
                              '7':'42_text',
                              '8':'43_text',
                              '9':'44_text',
                              '10':'45_text',
                              '11':'46_text',
                              '12':'47_text'
                             },inplace=True
                    )

# Show
surveys_f.head(2)

,ID,territory,36_text,37_text,38_text,39_text,40_text,41_text,42_text,43_text,44_text,45_text,46_text,47_text
3,4,TA_Miramar,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo
7,8,TA_Miramar,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo


## Assign values to survey's answers

In [9]:
# Mapping dictionary (Assigns value to answer)
mapping_dct = {"Totalmente en desacuerdo": -2,
               "En desacuerdo": -1,
               "Ni de acuerdo, ni en desacuerdo": 0,
               "De acuerdo": 1,
               "Totalmente de acuerdo": 2
}

# Mapping dictionary for question #38. (¿Se te hace largo y aburrido el recorrido?), 
# where answers pivoting towards "yes" indicate ausence of the physical variable (intersections density)
# and answers pivoting towards "no" indicate presence of the physical variable (intersecions density)
inverse_mapping_dct = {"Totalmente en desacuerdo": 2,
                       "En desacuerdo": 1,
                       "Ni de acuerdo, ni en desacuerdo": 0,
                       "De acuerdo": -1,
                       "Totalmente de acuerdo": -2
}

# Apply mappings
question_numbers = ['36','37','38','39','40','41','42','43','44','45','46','47']
for question in question_numbers:
    # Apply regular mapping to all except question 38
    if question != '38':
        surveys_f[question+'_value'] = surveys_f[question+'_text'].map(mapping_dct)
    # Apply inverse mapping to question 38
    elif question == '38':
        surveys_f[question+'_value'] = surveys_f[question+'_text'].map(inverse_mapping_dct)

# Reorder columns to visualize
ordered_cols = []
for question in question_numbers:
    ordered_cols.append(question+'_text')
    ordered_cols.append(question+'_value')
surveys_f = surveys_f[['ID','territory']+ordered_cols]

# Show
surveys_f.head(5)

,ID,territory,36_text,36_value,37_text,37_value,38_text,38_value,39_text,39_value,...,43_text,43_value,44_text,44_value,45_text,45_value,46_text,46_value,47_text,47_value
3,4,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,...,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2
7,8,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,...,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2
9,10,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,...,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2
10,11,TA_Miramar,Totalmente de acuerdo,2,Totalmente de acuerdo,2,Totalmente en desacuerdo,2,Totalmente en desacuerdo,-2,...,Totalmente en desacuerdo,-2,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2
11,12,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,...,"Ni de acuerdo, ni en desacuerdo",0,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2


In [10]:
for col in ordered_cols:
    test = surveys_f.loc[surveys_f[col].isna()]
    if len(test)>0:
        print(f"Nans in {col}.")
    else:
        print(f"No nans in col {col}.")

No nans in col 36_text.
No nans in col 36_value.
No nans in col 37_text.
No nans in col 37_value.
No nans in col 38_text.
No nans in col 38_value.
No nans in col 39_text.
No nans in col 39_value.
No nans in col 40_text.
No nans in col 40_value.
No nans in col 41_text.
No nans in col 41_value.
No nans in col 42_text.
No nans in col 42_value.
No nans in col 43_text.
No nans in col 43_value.
No nans in col 44_text.
No nans in col 44_value.
No nans in col 45_text.
No nans in col 45_value.
No nans in col 46_text.
No nans in col 46_value.
No nans in col 47_text.
No nans in col 47_value.


## Calculate average by question, by territory

In [11]:
question_summary_df = pd.DataFrame()

for territory in list(surveys_f.territory.unique()):

    # Separate by territory
    surveys_f_territory = surveys_f.loc[surveys_f.territory==territory].copy()
    
    i = 0
    for question in question_numbers:
        # Calculate average for current question in current territory
        question_avg = surveys_f_territory[question+'_value'].mean()
    
        # Register average for current question in current territory
        question_summary_df.loc[i,'question'] = question
        question_summary_df.loc[i, territory] = question_avg
    
        i+=1

question_summary_df

,question,TA_Miramar,TA_Providencia,TA_Colinas
0,36,0.620155,0.597222,0.642857
1,37,-0.147287,0.902778,0.000000
2,38,1.240310,1.430556,1.642857
3,39,0.751938,1.069444,0.500000
4,40,1.139535,1.013889,0.571429
5,41,0.441860,0.486111,-0.214286
6,42,-0.255814,0.194444,0.214286
7,43,-0.387597,1.236111,0.642857
8,44,-0.100775,1.458333,0.785714
9,45,-0.829457,-1.569444,-1.071429


## Calculate betas

In [12]:
betas_df = pd.DataFrame()

questions_variables_dct = {'population_dens':['36'],
                           'intersections_dens':['37','38','39'],
                           'land_use':['40','41','42'],
                           'vegetation':['43','44'],
                           'slope':['45'],
                           'sidewalk':['46','47']
                          }

# Calculate betas by variable for each territory
for territory in list(surveys_f.territory.unique()):
    
    # Each territory's data is registered as a dict. in rows
    rows = []
    
    for variable, question_list in questions_variables_dct.items():
        
        # List all question average values
        var_values = []
        for question in question_list:
            idx_question = question_summary_df.question==question
            question_avg = question_summary_df.loc[idx_question,territory].values[0]
            var_values.append(question_avg)

        # Get an average of those averages
        variable_avg = sum(var_values) / len(var_values)

        # Register in rows
        rows.append({'variable': variable, territory: variable_avg})
        
    # Register territory data in betas_df
    territory_df = pd.DataFrame(rows)
    if betas_df.empty:
        betas_df = territory_df
    else:
        betas_df = pd.merge(betas_df, territory_df, on='variable')

betas_df

,variable,TA_Miramar,TA_Providencia,TA_Colinas
0,population_dens,0.620155,0.597222,0.642857
1,intersections_dens,0.614987,1.134259,0.714286
2,land_use,0.441860,0.564815,0.190476
3,vegetation,-0.244186,1.347222,0.714286
4,slope,-0.829457,-1.569444,-1.071429
5,sidewalk,-0.585271,0.513889,-0.178571
